In [0]:
import scandir
import os, sys
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import dill
import random
import vcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri as p2r
p2r.activate()
r = ro.r
import shutil
from utils import read_df, save_df
from pathlib import Path, PurePath
from ipyparallel import Client
from collections import Counter, defaultdict, namedtuple, OrderedDict
from scipy.stats import mannwhitneyu, ks_2samp
import tables
import ujson
import pickle

In [0]:
rc = Client(profile="sge")

In [0]:
dview = rc[:]
lview = rc.load_balanced_view()
len(dview)

In [0]:
analysis_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/"

In [0]:
phenos = ["mass", "tdt", "pd"]

In [0]:
hmeans = {"mass": os.path.join(analysis_dir, "mcmc_mass_hmean.txt"),
         "tdt": os.path.join(analysis_dir, "mcmc_tdt_hmean.txt"),
         "pd": os.path.join(analysis_dir, "mcmc_pd_hmean.txt")}

quants = {"mass": os.path.join(analysis_dir, "mass_quant.txt"),
         "tdt": os.path.join(analysis_dir, "tdt_quant.txt"),
         "pd": os.path.join(analysis_dir, "pd_quant.txt")}

In [0]:
def read_hmean(key):
    return pd.read_csv(hmeans[key], sep="\t", index_col=0).apply(pd.to_numeric, errors="ignore")

In [0]:
def read_quant(key):
    return pd.read_csv(quants[key], sep="\t", index_col=0).apply(pd.to_numeric, errors="ignore")

In [0]:
mass_hmean = read_hmean("mass")
pd_hmean = read_hmean("pd")
tdt_hmean = read_hmean("tdt")


In [0]:
mass_quant = read_quant("mass")
pd_quant = read_quant("pd")
tdt_quant = read_quant("tdt")

In [0]:
mass_sig = mass_hmean[mass_hmean.postrb_hmean > mass_quant.ix['cutoff'].values[0]]
mass_relaxed = mass_hmean[mass_hmean.postrb_hmean > mass_quant.ix['relaxed_cutoff'].values[0]]

In [0]:
mass_upper20 = mass_hmean.sort_values('postrb_hmean', ascending=False).head(20)
pd_upper20 = pd_hmean.sort_values('postrb_hmean', ascending=False).head(20)


tdt_upper20 = tdt_hmean.sort_values('postrb_hmean', ascending=False).head(20)
tdt_lower20 = tdt_hmean.sort_values('postrb_hmean', ascending=False).tail(20)

In [0]:
sns.distplot(mass_sig.postrb_hmean);

In [0]:
sns.distplot(pd_sig.postrb_hmean);

In [0]:
sns.distplot(tdt_sig.postrb_hmean);

In [0]:
gt_base_df = read_df(analysis_dir, 'gt_base_df')

In [0]:
pops = sorted(set([x.split("_")[0] for x in gt_base_df.index]))

In [0]:
def count_genotypes(snp):
    counts = Counter()
    for gt in snp:
        try:
            float(gt) #if gt is nan
        except:
            counts[gt[0]]+=1
            counts[gt[-1]]+=1
    return sorted(counts.items(), key=operator.itemgetter(1))

In [0]:
gt_base_df['population'] = gt_base_df.apply(lambda x: x.name.split("_")[0], axis=1)

In [0]:
pop_allele_data = {}

def add_allele_freq(gt_list):
    data = gt_list
    ret = OrderedDict()
    if len(gt_list) == 2:
        total = data[0][1]+data[1][1]
        ret[data[0][0]] = [data[0][1], data[0][1]/total]
        ret[data[1][0]] = [data[1][1], data[1][1]/total]
    else:
        ret[data[0][0]] = [data[0][1], 1.0]
    return ret

for group, data in gt_base_df.groupby('population'):
    data = data.drop('population', axis=1)
    print(group, data.shape)
    gt = data.apply(count_genotypes).apply(add_allele_freq)
    pop_allele_data[group] = gt.to_dict()

In [0]:
pimass_gt = read_df(analysis_dir, '_pimass_gt').replace("NA", np.nan)

In [0]:
pimass_gt.head()

In [0]:
pd.DataFrame(pop_allele_data['VA1'])

In [0]:
gt_counts = gt_base_df.apply(count_genotypes)

In [0]:
gt_counts_af = gt_counts.apply(add_allele_freq)

In [0]:
gt_counts_af.head()

In [0]:
dview['pimass_gt'] = pimass_gt

In [0]:
dview['pops'] = pops

In [0]:
dview['analysis_dir'] = analysis_dir

In [0]:
%px import os, pickle, traceback

In [0]:
with open(os.path.join(analysis_dir, "pop_allele_data.pkl"), "wb") as o:
    pickle.dump(pop_allele_data, o, pickle.HIGHEST_PROTOCOL)

In [0]:
%px pop_allele_data = pickle.load(open(os.path.join(analysis_dir, "pop_allele_data.pkl"), "rb"))

In [0]:
def compute_obs_heterozygosity(snp):
    het = 0
    total = 0
    for gt in snp:
        if gt[0] != gt[-1]:
            het += 1
        if gt[1] == "/" or gt[1] == "|":
            total += 1
    return het/total

def compute_exp_heterozygosity(snp):
    het = 0
    total = 0
    c = Counter()
    for gt in snp:
        c[gt[0]] += 1
        c[gt[-1]] += 1
    total = np.sum(list(c.values()))
    He = 2
    for a in c:
        He *= (c[a]/total)
    return He

In [0]:
het_bins = np.linspace(0,1,20)

In [0]:
He = gt_base_df.drop("population", axis=1).apply(compute_exp_heterozygosity)
He = pd.DataFrame(He, columns=["He"])
He['rs'] = He.index

In [0]:
Ho = gt_base_df.drop("population", axis=1).apply(compute_obs_heterozygosity)
Ho = pd.DataFrame(Ho, columns=["Ho"])
Ho['rs'] = Ho.index

In [0]:
tdt_ho = tdt_hmean.join(Ho, on="rs", rsuffix="_ho", how="inner")
mass_ho = mass_hmean.join(Ho, on="rs", rsuffix="_ho", how="inner")
pd_ho = pd_hmean.join(Ho, on="rs", rsuffix="_ho", how="inner")

tdt_he = tdt_hmean.join(He, on="rs", rsuffix="_he", how="inner")
mass_he = mass_hmean.join(He, on="rs", rsuffix="_he", how="inner")
pd_he = pd_hmean.join(He, on="rs", rsuffix="_he", how="inner")

In [0]:
tdt_ho['het_bin'] = np.digitize(tdt_ho.Ho, het_bins)
mass_ho['het_bin'] = np.digitize(mass_ho.Ho, het_bins)
pd_ho['het_bin'] = np.digitize(pd_ho.Ho, het_bins)

tdt_he['het_bin'] = np.digitize(tdt_he.He, het_bins)
mass_he['het_bin'] = np.digitize(mass_he.He, het_bins)
pd_he['het_bin'] = np.digitize(pd_he.He, het_bins)

In [0]:
plt.scatter(tdt_he.postrb_hmean, tdt_he.He)
plt.xlabel("PIP")
plt.ylabel(r"$H_{exp}$")
plt.title("TDT")
plt.show()

plt.scatter(mass_he.postrb_hmean, tdt_he.He)
plt.xlabel("PIP")
plt.ylabel(r"$H_{exp}$")
plt.title("Mass")
plt.show()

plt.scatter(pd_he.postrb_hmean, tdt_he.He)
plt.xlabel("PIP")
plt.ylabel(r"$H_{exp}$")
plt.title("PD")
plt.show()

In [0]:
plt.scatter(tdt_ho.postrb_hmean, tdt_ho.Ho)
plt.xlabel("PIP")
plt.ylabel(r"$H_{obs}$")
plt.title("TDT")
plt.show()

plt.scatter(mass_ho.postrb_hmean, tdt_ho.Ho)
plt.xlabel("PIP")
plt.ylabel(r"$H_{obs}$")
plt.title("Mass")
plt.show()

plt.scatter(pd_ho.postrb_hmean, tdt_ho.Ho)
plt.xlabel("PIP")
plt.ylabel(r"$H_{obs}$")
plt.title("PD")
plt.show()

In [0]:
@lview.remote()
def do_pairwise(sig_list):
    import numpy as np
    import traceback
    ret = []
    for i, snp in enumerate(sig_list):
        snp_i = snp
        minor_allele_i = pimass_gt.ix[snp_i, "minor"]
        for j in range(i):
            snp_j = sig_list[j]
            minor_allele_j = pimass_gt.ix[snp_j, "minor"]
            in_prods = []
            freqs = {snp_i: [], snp_j: []}
            for p in pops:
                paf_i = paf_j = 0.0
                try:
                    paf_i = pop_allele_data[p][snp][minor_allele_i][1]
                    paf_j = pop_allele_data[p][snp_j][minor_allele_j][1]
                    freqs[snp_i].append(paf_i)
                    freqs[snp_j].append(paf_j)
                    in_prods.append(paf_i * paf_j)
                    avg_in_prod = np.mean(in_prods)
                    freqs_avg = {k: np.mean(freqs[k]) for k in freqs}
                    across_freqs = list(freqs_avg.values())
                    across_prod = across_freqs[0] * across_freqs[1]
                    ret.append(avg_in_prod-across_prod)
                except KeyError as e:
                    pass #in case allele is fixed in pop
                except:
                    traceback.print_exc()
    return ret

In [0]:
def get_nulls(n, sig_df, df):
    unassoc = df.drop(sig_df.index)
    het_bin_counts = df.ix[sig_df.index]['het_bin'].value_counts()
    het_bins = het_bin_counts.index.tolist()
    unassoc = unassoc[unassoc.het_bin.isin(het_bins)]
    data = []
    for i in range(n):
        inner = []
        for het_bin, het_count in het_bin_counts.iteritems():
            inner.extend(unassoc[unassoc.het_bin == het_bin].rs.sample(het_count).tolist())
        data.append(inner)
    return data, het_bins

In [0]:
dview['get_nulls'] = get_nulls

In [0]:
gwas_upper_D = do_pairwise(tdt_upper20.rs.tolist()).r

In [0]:
gwas_lower_D = do_pairwise(tdt_lower20.rs.tolist()).r

In [0]:
tdt_lower20 = tdt_he[tdt_he.het_bin.isin(range(4,9))].sort_values("postrb_hmean", ascending=False).tail(20)

In [0]:
nulls_upper, upper_sig_het_bins = get_nulls(1000, tdt_upper20, tdt_he)
nulls_lower, lower_sig_het_bins = get_nulls(1000, tdt_lower20, tdt_he)

In [0]:
het_bins

In [0]:
nulls_upper_D = []
for i, n in enumerate(nulls_upper):
    if i % 100 == 0:
        print(i)
    nulls_upper_D.append(do_pairwise(n))
    
nulls_lower_D = []
for i, n in enumerate(nulls_lower):
    if i % 100 == 0:
        print(i)
    nulls_lower_D.append(do_pairwise(n))

In [0]:
@lview.remote()
def ks_test(arr1, arr2):
    from scipy.stats import ks_2samp
    return ks_2samp(arr1, arr2)

@lview.remote()
def mwu_test(arr1, arr2):
    from scipy.stats import mannwhitneyu
    return mannwhitneyu(arr1, arr2)

In [0]:
nulls_upper_D = [x.r for x in nulls_upper_D]
nulls_lower_D = [x.r for x in nulls_lower_D]

In [0]:
ks_upper = [ks_test(gwas_upper_D, x) for x in nulls_upper_D]
ks_lower = [ks_test(gwas_lower_D, x) for x in nulls_lower_D]

In [0]:
gwas_null_ks_upper = [x.r for x in ks_upper]
gwas_null_ks_lower = [x.r for x in ks_lower]

In [0]:
gwas_null_pvals_upper = []
for stat, pval in gwas_null_ks_upper:
    gwas_null_pvals_upper.append(pval)
    
gwas_null_pvals_lower = []
for stat, pval in gwas_null_ks_lower:
    gwas_null_pvals_lower.append(pval)

In [0]:
nulls2_upper = get_nulls(1000, tdt_upper20, tdt_he)
nulls3_upper = get_nulls(1000, tdt_upper20, tdt_he)

nulls2_lower = get_nulls(1000, tdt_lower20, tdt_he)
nulls3_lower = get_nulls(1000, tdt_lower20, tdt_he)

In [0]:
n2_upper = [do_pairwise(x) for x in nulls2_upper[0]]
n3_upper = [do_pairwise(x) for x in nulls3_upper[0]]

n2_lower = [do_pairwise(x) for x in nulls2_lower[0]]
n3_lower = [do_pairwise(x) for x in nulls3_lower[0]]

In [0]:
n2_r_upper = [x.r for x in n2_upper]
n3_r_upper = [x.r for x in n3_upper]

n2_r_lower = [x.r for x in n2_lower]
n3_r_lower = [x.r for x in n3_lower]

In [0]:
null_null_pvals_upper = []
for x, y in zip(n2_r_upper, n3_r_upper):
    stat, p = ks_2samp(x, y)
    null_null_pvals_upper.append(p)
    
null_null_pvals_lower = []
for x, y in zip(n2_r_lower, n3_r_lower):
    stat, p = ks_2samp(x, y)
    null_null_pvals_lower.append(p)

In [0]:
ks_2samp(gwas_null_pvals_upper, null_null_pvals_upper)

In [0]:
ks_2samp(gwas_null_pvals_lower, null_null_pvals_lower)

In [0]:
plt.scatter(tdt_he.ix[tdt_lower20.index].postrb_hmean, tdt_he.ix[tdt_lower20.index].He)

In [0]:
plt.scatter(tdt_he.ix[tdt_upper20.index].postrb_hmean, tdt_he.ix[tdt_upper20.index].He)